# Deep Neural Network for Exoplanet Discovery Method Classification
## Identifying Patterns in Exoplanet Characteristics to Determine Discovery Methods

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import optuna as opt

/Users/kevin/anaconda3/envs/tf_env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data

In [2]:
# data preprocessed in another jupytr nb 

composite_preprocessed = pd.read_csv('Composite_preprocessed_NO_MV_BALANCED.csv')
composite_preprocessed.head()

,Number of Stars,Number of Planets,Number of Moons,Galactic Latitude [deg],Galactic Longitude [deg],Ecliptic Latitude [deg],Ecliptic Longitude [deg],Number of Photometry Time Series,Number of Radial Velocity Time Series,Number of Stellar Spectra Measurements,Number of Emission Spectroscopy Measurements,Number of Transmission Spectroscopy Measurements,Circumbinary Flag,Controversial Flag,Discovery Year,Detected by Transits
0,3.094076,-0.666894,0.0,2.424559,1.234306,-0.145901,-0.624689,0.613405,1.731519,-0.261600,-0.012466,-0.044364,0,0,2007,0
1,-0.256668,-0.666894,0.0,1.186720,-0.283545,1.148661,-0.992906,0.613405,0.729626,-0.261600,-0.012466,-0.044364,0,0,2009,0
2,-0.256668,-0.666894,0.0,-0.877523,-0.306068,0.308947,-2.327878,0.613405,0.729626,-0.261600,-0.012466,-0.044364,0,0,2008,0
3,-0.256668,0.216988,0.0,1.382856,-0.669803,0.872499,-0.152934,0.613405,3.735304,0.591749,-0.012466,-0.044364,0,0,2002,0
4,6.444820,-0.666894,0.0,0.261241,-0.531444,1.023143,0.855489,0.613405,3.735304,2.298449,-0.012466,-0.044364,0,0,1996,0


### 2 possible results ; Either the exoplanet was discovered by transits (1) or it wasn't (0) ; binary classification
### so we have 1 neuron in the output layer  
### We will use the features from the XGBoost refined features model 
### This ensures consistency, reduces complexity, and leverages the feature selection process that has already been validated with my refined features XGBoost model
### This would also allow for a fair comparison between models and is likely to result in better performance for the DNN

## Remove unimportant features
### From the logistic regression model, 'Number of Moons' can be excluded because it does not provide value to the model

In [3]:
remove = ['Number of Radial Velocity Time Series',
          'Number of Stellar Spectra Measurements',
          'Controversial Flag',
          'Circumbinary Flag',
          'Number of Moons']

composite_unnecessary_removed = composite_preprocessed.drop(remove, axis=1)

## Train Test Split

In [4]:
# we are trying to predict whether an exoplanet has been detected by transits (0 or 1)
# target variable is if the exoplanet was discovered by transits

targets = composite_unnecessary_removed['Detected by Transits']

# training features are all variables except the targets

features = composite_unnecessary_removed.drop(['Detected by Transits'], axis=1)

## Observe Feature Correlations

In [5]:
features.corr()

,Number of Stars,Number of Planets,Galactic Latitude [deg],Galactic Longitude [deg],Ecliptic Latitude [deg],Ecliptic Longitude [deg],Number of Photometry Time Series,Number of Emission Spectroscopy Measurements,Number of Transmission Spectroscopy Measurements,Discovery Year
Number of Stars,1.000000,0.109074,-0.044988,0.104236,-0.087528,-0.069450,0.057734,-0.003200,-0.002096,-0.147575
Number of Planets,0.109074,1.000000,-0.055511,0.008589,0.026680,-0.023926,-0.002505,0.057797,0.076409,-0.072274
Galactic Latitude [deg],-0.044988,-0.055511,1.000000,-0.019213,0.463063,0.174632,-0.090369,-0.025691,-0.008918,0.031820
Galactic Longitude [deg],0.104236,0.008589,-0.019213,1.000000,-0.657044,-0.408779,0.131333,-0.008283,-0.026088,-0.106558
Ecliptic Latitude [deg],-0.087528,0.026680,0.463063,-0.657044,1.000000,0.498442,-0.222283,-0.006864,0.025212,0.081113
Ecliptic Longitude [deg],-0.069450,-0.023926,0.174632,-0.408779,0.498442,1.000000,-0.119612,0.013811,0.031078,0.058041
Number of Photometry Time Series,0.057734,-0.002505,-0.090369,0.131333,-0.222283,-0.119612,1.000000,-0.004172,-0.014848,-0.252983
Number of Emission Spectroscopy Measurements,-0.003200,0.057797,-0.025691,-0.008283,-0.006864,0.013811,-0.004172,1.000000,-0.000553,0.000791
Number of Transmission Spectroscopy Measurements,-0.002096,0.076409,-0.008918,-0.026088,0.025212,0.031078,-0.014848,-0.000553,1.000000,-0.015425
Discovery Year,-0.147575,-0.072274,0.031820,-0.106558,0.081113,0.058041,-0.252983,0.000791,-0.015425,1.000000


### Moderate correlation between Ecliptic Longtiude aand Ecliptic Latitude

## Split data

In [6]:
# Splitting dataset into training, testing and validation addresses overfitting
# shuffling is necessary to remove dependencies that come from order of data

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, targets, train_size = 0.8, random_state = 42, shuffle=True)

# tensorflow models expect data in the form of a np array; NOT a pd dataframe 
# so we have to convert

x_train_nparr, x_test_nparr, y_train_nparr, y_test_nparr = np.array(x_train), np.array(x_test), np.array(y_train), np.array(y_test)


## DNN Configuration

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# we use optuna for hyperparameter optimization
from optuna.integration import TFKerasPruningCallback

### Create DNN

In [8]:
# Initializes a Sequential Model (linear stack of layers)
# Simplest way to build a model in Keras, where you can just keep adding layers sequentially

input_size = x_train.shape[1] # 10 input features in our data 
# hidden_layer_size = 50 # start off with 50 neurons per hidden layer
output_size = 1 # because 2 possible classifications denoted by 0 and 1 in a single output; DNN calculates probability the exoplanet is 0 or 1 and the higher probability is the output 

# dropout_rate = 0.05 # dropout rate we will use in the model as a measure to prevent overfitting

def create_dnn(trial):

    # since we use optuna, we have to toss in the values we want it to test during the optimization process

    # units = neurons in the hidden later, 32 to 512 jumping 32 at a time 
    units = trial.suggest_int('units', 10, 100, step=5)

    # dropout rate is self explanatory
    dropout_rate = trial.suggest_float('dropout', 0.0, 0.5, step=0.05)
    
    dnn_model = Sequential([
    # input layer for the input data; 10 features so 10 inputs for the first dense layer 
    # in a dense layer, each feature becomes an input to the next layer, which consists of neurons
    # the line below defines the input layer and creates the first dense layer consisting of 50 neurons
    # we use Rectified Linear Unit (ReLU) as the activation function because ReLU helps mitigate the vanishing gradient problem
# and allows the model to learn complex paterns
    tf.keras.layers.Dense(units, activation='relu', input_shape = (input_size,)),

# add a dropout layer with dropout rate of 0.1 
# This essentially just means 10% of the neurons will be randomly set to 0 during training
# Dropout is a regularization technique used to prevent overfitting 
# This helps prevent the model from overfitting by randomly dropping neurons during training
    tf.keras.layers.Dropout(dropout_rate),
    
# we only use 1 hidden layer of neurons because 2 lowers validation accuracy;
    
    # output layer with 1 neuron for binary classification
    tf.keras.layers.Dense(output_size, activation='sigmoid')
    ])
# next, choose the optimizer and loss function
# adam is usually the most optimal
# loss will be binary_crossentropy for binary classification
# the metric we care about is accuracy
    dnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return dnn_model

def objective(trial):
    model = create_dnn(trial)


    # suggest number of epochs and batch size
    epochs = trial.suggest_int('epochs', 5, 50, step=5)
    batch_size = trial.suggest_int('batch_size', 5,30,step=5)
    training = model.fit(x_train_nparr, 
            y_train_nparr, 
            epochs=epochs, 
            batch_size=batch_size, 
            validation_split=0.1, 
            callbacks=[TFKerasPruningCallback(trial, 'val_accuracy')], 
            verbose=2
    )
    val_accuracy = max(training.history['val_accuracy'])
    return val_accuracy

# create a study and optimize the objective function
study = opt.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

(f'Best trial: {study.best_trial.value}', f'Best hyperparameters: {study.best_trial.params}')


[I 2024-08-02 16:27:49,255] A new study created in memory with name: no-name-e81dd2ef-d144-4400-8793-5c417d695520
2024-08-02 16:27:49.281548: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-02 16:27:49.282749: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


Train on 4633 samples, validate on 515 samples
Epoch 1/10
4633/4633 - 3s - loss: 20.2072 - accuracy: 0.5534 - val_loss: 2.0505 - val_accuracy: 0.5165
Epoch 2/10
4633/4633 - 2s - loss: 9.5028 - accuracy: 0.6378 - val_loss: 0.6057 - val_accuracy: 0.8777
Epoch 3/10
4633/4633 - 2s - loss: 4.2847 - accuracy: 0.6980 - val_loss: 0.4817 - val_accuracy: 0.8447
Epoch 4/10
4633/4633 - 2s - loss: 1.4372 - accuracy: 0.7578 - val_loss: 0.8533 - val_accuracy: 0.6641
Epoch 5/10
4633/4633 - 2s - loss: 0.4921 - accuracy: 0.7984 - val_loss: 0.3630 - val_accuracy: 0.8738
Epoch 6/10
4633/4633 - 2s - loss: 0.4619 - accuracy: 0.7975 - val_loss: 0.6768 - val_accuracy: 0.5184
Epoch 7/10
4633/4633 - 2s - loss: 0.4502 - accuracy: 0.8047 - val_loss: 0.3477 - val_accuracy: 0.8777
Epoch 8/10
4633/4633 - 2s - loss: 0.4298 - accuracy: 0.8252 - val_loss: 0.4742 - val_accuracy: 0.8835
Epoch 9/10
4633/4633 - 2s - loss: 0.4230 - accuracy: 0.8206 - val_loss: 0.3697 - val_accuracy: 0.8932
Epoch 10/10
4633/4633 - 2s - loss:

[I 2024-08-02 16:28:13,082] Trial 0 finished with value: 0.893203854560852 and parameters: {'units': 25, 'dropout': 0.05, 'epochs': 10, 'batch_size': 5}. Best is trial 0 with value: 0.893203854560852.


Train on 4633 samples, validate on 515 samples
Epoch 1/15
4633/4633 - 3s - loss: 30.2444 - accuracy: 0.5200 - val_loss: 3.0771 - val_accuracy: 0.5301
Epoch 2/15
4633/4633 - 2s - loss: 8.5295 - accuracy: 0.5763 - val_loss: 1.1060 - val_accuracy: 0.5670
Epoch 3/15
4633/4633 - 2s - loss: 1.5059 - accuracy: 0.7116 - val_loss: 0.4257 - val_accuracy: 0.8641
Epoch 4/15
4633/4633 - 2s - loss: 0.5508 - accuracy: 0.7695 - val_loss: 0.4197 - val_accuracy: 0.8874
Epoch 5/15
4633/4633 - 2s - loss: 0.5149 - accuracy: 0.7803 - val_loss: 1.0572 - val_accuracy: 0.5612
Epoch 6/15
4633/4633 - 2s - loss: 0.5001 - accuracy: 0.7742 - val_loss: 0.4120 - val_accuracy: 0.8816
Epoch 7/15
4633/4633 - 2s - loss: 0.4576 - accuracy: 0.8085 - val_loss: 0.4134 - val_accuracy: 0.8874
Epoch 8/15
4633/4633 - 2s - loss: 0.4319 - accuracy: 0.8176 - val_loss: 0.3991 - val_accuracy: 0.7961
Epoch 9/15
4633/4633 - 2s - loss: 0.4397 - accuracy: 0.8172 - val_loss: 0.4201 - val_accuracy: 0.7709
Epoch 10/15
4633/4633 - 2s - loss:

[I 2024-08-02 16:28:47,371] Trial 1 finished with value: 0.8970873951911926 and parameters: {'units': 60, 'dropout': 0.1, 'epochs': 15, 'batch_size': 5}. Best is trial 1 with value: 0.8970873951911926.


...

Train on 4633 samples, validate on 515 samples
Epoch 1/50
4633/4633 - 3s - loss: 24.4033 - accuracy: 0.5292 - val_loss: 0.9078 - val_accuracy: 0.5417
Epoch 2/50


[I 2024-08-02 16:49:57,838] Trial 45 pruned. Trial was pruned at epoch 1.


Train on 4633 samples, validate on 515 samples
Epoch 1/50


/Users/kevin/anaconda3/envs/tf_env/lib/python3.7/site-packages/optuna_integration/tfkeras.py:53: UserWarning: The metric 'val_accuracy' is not in the evaluation logs for pruning. Please make sure you set the correct metric name.
  warnings.warn(message)
[W 2024-08-02 16:49:59,494] Trial 46 failed with parameters: {'units': 85, 'dropout': 0.0, 'epochs': 50, 'batch_size': 5} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/kevin/anaconda3/envs/tf_env/lib/python3.7/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/0h/qrfpw0p522n_hd8ccn5ffyvh0000gn/T/ipykernel_54356/401901747.py", line 59, in objective
    verbose=2
  File "/Users/kevin/anaconda3/envs/tf_env/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training.py", line 728, in fit
    use_multiprocessing=use_multiprocessing)
  File "/Users/kevin/anaconda3/envs/tf_env/lib/python3.7/site-packages/t

KeyboardInterrupt: 

### Extract the best hyperparameters determined by Optuna

In [ ]:
best_params = study.best_trial.params

### Train the model now using the best hyperparameters

In [ ]:
best_dnn = Sequential([
    
    tf.keras.layers.Dense(units= best_params['units'], activation='relu', input_shape = (input_size,)),

    tf.keras.layers.Dropout(best_params['dropout']),
    
    tf.keras.layers.Dense(output_size, activation='sigmoid')
    ])

best_dnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Fit the best model

In [ ]:
best_dnn_model = best_dnn.fit(
        x_train_nparr, 
        y_train_nparr, 
        epochs=best_params['epochs'], 
        batch_size=best_params['batch_size'], 
        validation_split=0.1, 
        verbose=2
    )

## Model Predictions

In [ ]:
# Here we actually make the predictions on the test set
# model outputs the probability for the positive class
# also convert the probabilities to binary class predictions with a threshold of 0.5

y_pred_prob = best_dnn_model.predict(x_test_nparr)

## Raw probabilities for the positive class

In [ ]:
# We use these probabilities for calculating the ROC-AUC score and other performance metrics that require probabilities
# ravel() flattens the matrix to a 1D array of probabilities

y_pred = (y_pred_prob > 0.5).astype("int32")

## Evaluate Model performance

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
import matplotlib.pyplot as plt

In [ ]:
('DNN Accuracy on Test Set:', accuracy_score(y_test_nparr, y_pred)),
('DNN Classification Report on Test Set:\n', classification_report(y_test_nparr, y_pred)),
('DNN ROC AUC Score on Test Set:', roc_auc_score(y_test_nparr, y_pred_prob))

## Plot Validation and Training Metrics for visual presentation 

In [ ]:
plt.figure(figsize=(10,6))

# training accuracy over epochs
plt.plot(best_dnn_model.history['accuracy'])

# validation accuracy over epochs
plt.plot(best_dnn_model.history['val_accuracy'])

plt.title('Exoplanet DNN Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Display loss separately 
# Loss is a fundamental measure of how well a machine learning model's predictions match the actual outcomes
# Loss quantifies the difference between pred values and actual values 
# Goal of training a model is to MINIMIZE LOSS
# By minimizing loss we can improve the model's accuracy and predictive power
# Loss guides hyperparamter tuning

plt.figure(figsize=(10,6))

# training loss over epochs
plt.plot(best_dnn_model.history['loss'])

# validation loss over epochs
plt.plot(best_dnn_model.history['val_loss'])

plt.title('Exoplanet DNN Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()